<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <div class="crop">
            <img src="https://image4.owler.com/logo/rapido_owler_20190125_062402_original.png" style="height:75px; margin-left:0px" />
        </div>
    </div>
    <div style="width: 59%; display: inline-block">
            <h1  style="text-align: center">SRuD Circuit Breaker Setup</h1><br>
        <div style="width: 90%; text-align: center; display: inline-block;"><i>By:</i> <strong>Ritik Prakash</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="Enter Date" pubdate>28th March, 2023</time>
        </div>
    </div>
</div>

___
___

<div class="alert alert-block alert-danger">

# Data Configs
</div>

<div class="alert alert-block alert-warning">
    
### Enter Configs To Run The Notebook
Comment or Edit the changes in Similar Format
</div>

___

In [204]:
file_name = 'Kol_22_28_cb_recommendation.parquet'

___

<div class="alert alert-block alert-info">

# Importing Packages
</div>

In [205]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import ast
from datetime import datetime,timedelta,date
import glob
import json
import uuid
import time

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 200)

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
tqdm.pandas()

from h3 import h3

In [206]:
from pyhive import presto
conn = presto.connect(
    host='presto-gateway.serving.data.plectrum.dev',
    port=443,
    protocol='https',
    catalog='hive',
    username='ritik.prakash@rapido.bike')

___

<div class="alert alert-block alert-info">

# Preparing Data
</div>

___

In [207]:
hh_routes_data = pd.read_parquet(file_name)

In [208]:
hh_routes_data

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Kolkata,Santoshpur EM Bypass,Aliah University New Town,Friday,1700,1800,ALL,ALL
1,Kolkata,Sector 1,Aliah University New Town,Friday,1730,1830,ALL,ALL
2,Kolkata,NetajiNagar,Aliah University New Town,Friday,1700,1800,ALL,ALL
3,Kolkata,Baguiati,Aliah University New Town,Friday,1730,1830,ALL,ALL
4,Kolkata,Dum Dum Junction,Aliah University New Town,Friday,1715,1815,ALL,ALL
...,...,...,...,...,...,...,...,...
37649,Kolkata,Netaji Nagar H,baranagar,Wednesday,0930,1030,ALL,ALL
37650,Kolkata,Howrah Station,baranagar,Wednesday,0915,1015,ALL,ALL
37651,Kolkata,baranagar,baranagar,Wednesday,0945,1045,ALL,ALL
37652,Kolkata,Uttarpara H,baranagar,Wednesday,0945,1045,ALL,ALL


___

<div class="alert alert-block alert-info">

# Build Hot Hot Configs
</div>

In [209]:
city_name = hh_routes_data['city'].unique()[0]

In [210]:
query = """

        SELECT 
            service_detail_id
        FROM 
            datasets.service_mapping
        WHERE 
            city_display_name = '{city}' 
            AND service_level = 'Link'

        """.format(city = city_name)

service_mapping = pd.read_sql_query(query, conn).iloc[0]
city_service_detail_id = service_mapping[0]

/Users/apple/opt/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [211]:
city_service_detail_id

'5c124a1e0241913903a42fe1'

___

___

___

In [212]:
CITY = city_name                                    # FORMAT: 'CITY_NAME', As per record
START_DATE = '20230530'                             # FORMAT: 'YYYYMMDD'
END_DATE = '20230605'                               # FORMAT: 'YYYYMMDD'
SERVICE_DETAIL_ID = city_service_detail_id          # FORMAT: 'SERVICE_DETAIL_ID', As per record
CAPTAIN_PAYOUT = 0                                  # FORMAT: In Rupees
STRATEGY = ['mismatch','hot_cold_surge']

___

___

___

<div class="alert alert-block alert-warning">

### Format
</div>

In [213]:
sample = {
    "service_detail_id": "5c04f8eec922ad49db3b5ec2",
    "end_date": "20220401",
    "city": "Guwahati",
    "run_date": "20220331",
    "updated_at": 1645675999,
    "recommendation_date": "20220401",
    "pickup_cluster": "Paltan Bazaar",
    "drop_cluster": "Paltan Bazaar",
    "end_time": "1030",
    "type": "order_acceptance",
    "captain_payout": 23,
    "start_date": "20220401",
    "created_at": 1645675999,
    "period": "",
    "updated_by": "bhargav.k@rapido.bike",
    "start_time": "1000",
    "pickup_hex8":"88619aa4d9fffff",
    "drop_hex8":"88619aa4d9fffff",
    "start_distance":0,
    "end_distance":100,
    "eligible_strategy":'mismatch',
    "rule_id": "d2e843de-a3bd-4de9-a2e5-22e41953d8d0"
}

In [214]:
config_format = json.loads(json.dumps(sample))
config_format = pd.DataFrame(config_format, index=[0])

In [215]:
config_format = config_format.columns

In [216]:
config_format

Index(['service_detail_id', 'end_date', 'city', 'run_date', 'updated_at',
       'recommendation_date', 'pickup_cluster', 'drop_cluster', 'end_time',
       'type', 'captain_payout', 'start_date', 'created_at', 'period',
       'updated_by', 'start_time', 'pickup_hex8', 'drop_hex8',
       'start_distance', 'end_distance', 'eligible_strategy', 'rule_id'],
      dtype='object')

___

<div class="alert alert-block alert-warning">

### Generating Configs
</div>

In [217]:
configuration = hh_routes_data.copy()

In [218]:
configuration.head()

,city,pickup_cluster,drop_cluster,week_period,start_time,end_time,pickup_hex8,drop_hex8
0,Kolkata,Santoshpur EM Bypass,Aliah University New Town,Friday,1700,1800,ALL,ALL
1,Kolkata,Sector 1,Aliah University New Town,Friday,1730,1830,ALL,ALL
2,Kolkata,NetajiNagar,Aliah University New Town,Friday,1700,1800,ALL,ALL
3,Kolkata,Baguiati,Aliah University New Town,Friday,1730,1830,ALL,ALL
4,Kolkata,Dum Dum Junction,Aliah University New Town,Friday,1715,1815,ALL,ALL


In [219]:
configuration['service_detail_id'] = SERVICE_DETAIL_ID
configuration['start_date'] = START_DATE
configuration['end_date'] = END_DATE
configuration['run_date'] = date.today().strftime('%Y%m%d')
configuration['start_time'] = configuration['start_time']
configuration['created_at'] = int(time.time())
configuration['updated_at'] = int(time.time())
configuration['period'] = ''
configuration['captain_payout'] = CAPTAIN_PAYOUT
configuration['type'] = 'order_acceptance'
configuration['updated_by'] = 'ritik.prakash@rapido.bike'
configuration['city'] = CITY
configuration['start_distance'] = 0
configuration['end_distance'] = 100

In [220]:
def rule(configuration):
    rule_id = str(uuid.uuid4())
    return rule_id

configuration['rule_id'] = configuration.apply(rule,axis=1)

In [221]:
config_df = configuration.copy()

In [222]:
start_day = datetime.strptime(START_DATE, '%Y%m%d') 
end_day = datetime.strptime(END_DATE, '%Y%m%d')
day = timedelta(days=1)

In [223]:
cur_day = start_day

In [224]:
rec_df = pd.DataFrame()

In [225]:
while cur_day <= end_day:
    
    week_day = cur_day.strftime("%A")
    rec_date = cur_day.strftime("%Y%m%d")
    
    week_data = config_df[config_df['week_period']==week_day]
    week_data = week_data.assign(recommendation_date=rec_date)
    
    rec_df = pd.concat([rec_df,week_data]).reset_index(drop=True)

    cur_day = cur_day + day

In [226]:
config_df = rec_df.sort_values(['city','recommendation_date'])

In [227]:
config_df['start_date'] = config_df['recommendation_date'].copy()
config_df['end_date'] = config_df['recommendation_date'].copy()

In [228]:
all_configuration = pd.DataFrame()

for stat in STRATEGY:
    config_all_df = config_df.copy()
    config_all_df['eligible_strategy'] = stat
    all_configuration = pd.concat([all_configuration,config_all_df]).reset_index(drop=True)

In [229]:
all_configuration = all_configuration[config_format].sort_values(by=['recommendation_date', 
                                                             'start_time']).reset_index(drop=True)

In [230]:
all_configuration.loc[all_configuration.start_time == '2300', 'end_time'] = '2400'

In [231]:
all_configuration.shape

(75308, 22)

In [232]:
all_configuration.head()

,service_detail_id,end_date,city,run_date,updated_at,recommendation_date,pickup_cluster,drop_cluster,end_time,type,captain_payout,start_date,created_at,period,updated_by,start_time,pickup_hex8,drop_hex8,start_distance,end_distance,eligible_strategy,rule_id
0,5c124a1e0241913903a42fe1,20230530,Kolkata,20230529,1685363714,20230530,New Nawpara,Saltlake sector 5,0515,order_acceptance,0,20230530,1685363714,,ritik.prakash@rapido.bike,0415,ALL,ALL,0,100,mismatch,6163433c-9970-40fb-993c-129cc4adf5bd
1,5c124a1e0241913903a42fe1,20230530,Kolkata,20230529,1685363714,20230530,New Nawpara,Saltlake sector 5,0515,order_acceptance,0,20230530,1685363714,,ritik.prakash@rapido.bike,0415,ALL,ALL,0,100,hot_cold_surge,6163433c-9970-40fb-993c-129cc4adf5bd
2,5c124a1e0241913903a42fe1,20230530,Kolkata,20230529,1685363714,20230530,New Town CE Block,Saltlake sector 5,0545,order_acceptance,0,20230530,1685363714,,ritik.prakash@rapido.bike,0445,ALL,ALL,0,100,mismatch,f2cd8857-f3c7-45b8-97b4-3a5284eccce7
3,5c124a1e0241913903a42fe1,20230530,Kolkata,20230529,1685363714,20230530,Saltlake sector 5,Saltlake sector 5,0545,order_acceptance,0,20230530,1685363714,,ritik.prakash@rapido.bike,0445,ALL,ALL,0,100,mismatch,8499cee5-7513-498c-b517-723cbaba8f57
4,5c124a1e0241913903a42fe1,20230530,Kolkata,20230529,1685363714,20230530,New Town CE Block,Saltlake sector 5,0545,order_acceptance,0,20230530,1685363714,,ritik.prakash@rapido.bike,0445,ALL,ALL,0,100,hot_cold_surge,f2cd8857-f3c7-45b8-97b4-3a5284eccce7


In [233]:
all_configuration.dtypes

service_detail_id      object
end_date               object
city                   object
run_date               object
updated_at              int64
recommendation_date    object
pickup_cluster         object
drop_cluster           object
end_time               object
type                   object
captain_payout          int64
start_date             object
created_at              int64
period                 object
updated_by             object
start_time             object
pickup_hex8            object
drop_hex8              object
start_distance          int64
end_distance            int64
eligible_strategy      object
rule_id                object
dtype: object

___

___

In [234]:
all_configuration.shape

(75308, 22)

In [235]:
all_configuration['recommendation_date'].unique()

array(['20230530', '20230531', '20230601', '20230602', '20230603',
       '20230604', '20230605'], dtype=object)

In [68]:
# delhi = all_configuration.copy()

In [101]:
# hyd = all_configuration.copy()

In [134]:
# che = all_configuration.copy()

In [167]:
# blr = all_configuration.copy()

In [200]:
# jai = all_configuration.copy()

In [236]:
# kol = all_configuration.copy()

In [237]:
all_city = pd.concat([delhi,hyd,che,blr,jai,kol]).reset_index(drop=True)

In [243]:
final_ac = all_city.copy()

In [244]:
def rule(configuration):
    rule_id = str(uuid.uuid4())
    return rule_id

final_ac['rule_id'] = final_ac.apply(rule,axis=1)

In [245]:
final_ac

,service_detail_id,end_date,city,run_date,updated_at,recommendation_date,pickup_cluster,drop_cluster,end_time,type,captain_payout,start_date,created_at,period,updated_by,start_time,pickup_hex8,drop_hex8,start_distance,end_distance,eligible_strategy,rule_id
0,5da4660028af187d8d52cc3c,20230530,Delhi,20230529,1685362964,20230530,Rohini_del,DLF Phase-1,0730,order_acceptance,0,20230530,1685362964,,ritik.prakash@rapido.bike,0630,ALL,ALL,0,100,mismatch,171c789d-8165-4d9e-a854-bbe81daaed73
1,5da4660028af187d8d52cc3c,20230530,Delhi,20230529,1685362964,20230530,NCR N Okhla 2,Dwarka_Del,0730,order_acceptance,0,20230530,1685362964,,ritik.prakash@rapido.bike,0630,ALL,ALL,0,100,mismatch,554caf22-d98d-44c3-bbcd-5578b8bc77ab
2,5da4660028af187d8d52cc3c,20230530,Delhi,20230529,1685362964,20230530,Rohini_del,DLF Phase-1,0730,order_acceptance,0,20230530,1685362964,,ritik.prakash@rapido.bike,0630,ALL,ALL,0,100,hot_cold_surge,64db7300-ed44-4034-b708-ff8ff2a1f22a
3,5da4660028af187d8d52cc3c,20230530,Delhi,20230529,1685362964,20230530,NCR N Okhla 2,Dwarka_Del,0730,order_acceptance,0,20230530,1685362964,,ritik.prakash@rapido.bike,0630,ALL,ALL,0,100,hot_cold_surge,bb57d29d-58fc-404d-bbf0-a96858b80c8e
4,5da4660028af187d8d52cc3c,20230530,Delhi,20230529,1685362964,20230530,NCR F Sector 16,NCR N Gharoli,0745,order_acceptance,0,20230530,1685362964,,ritik.prakash@rapido.bike,0645,ALL,ALL,0,100,mismatch,b4dfce9f-9c42-4ecd-be9d-9e0197c628fd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2442353,5c124a1e0241913903a42fe1,20230605,Kolkata,20230529,1685363714,20230605,Sealdah,Taltala,2145,order_acceptance,0,20230605,1685363714,,ritik.prakash@rapido.bike,2045,ALL,ALL,0,100,hot_cold_surge,386783dd-e08a-49cf-90b1-314a4312314b
2442354,5c124a1e0241913903a42fe1,20230605,Kolkata,20230529,1685363714,20230605,Park Circus,Taltala,2145,order_acceptance,0,20230605,1685363714,,ritik.prakash@rapido.bike,2045,ALL,ALL,0,100,hot_cold_surge,ea5fee7d-bfc9-4b6d-bcd4-29000883391e
2442355,5c124a1e0241913903a42fe1,20230605,Kolkata,20230529,1685363714,20230605,Ballygunge,Taltala,2145,order_acceptance,0,20230605,1685363714,,ritik.prakash@rapido.bike,2045,ALL,ALL,0,100,hot_cold_surge,b6d70c4b-ccfc-4902-86a1-f1013ac3e501
2442356,5c124a1e0241913903a42fe1,20230605,Kolkata,20230529,1685363714,20230605,Beck Bagan,Taltala,2145,order_acceptance,0,20230605,1685363714,,ritik.prakash@rapido.bike,2045,ALL,ALL,0,100,hot_cold_surge,b3aa3d78-73ee-455c-bb06-1c428c19e4c5


In [251]:
final_ac['rule_id'].nunique()

2442358

___

___

<div class="alert alert-block alert-danger">

# Upload the Configs
</div>

___

### 1. Write configurations as parquet.


In [249]:
configurations_directory = '/tmp/circuit_breaker_configurations/'

!rm -R {configurations_directory}
!mkdir {configurations_directory}

final_ac.to_parquet(configurations_directory, partition_cols=['run_date', 'city', 'recommendation_date'])

___

### 2. Open Temp File


In [250]:
!open /tmp/

___

### 3. Uncomment the line below if you want to upload recommendations to gcs to update the Presto table


In [166]:
# !gsutil -q -m rsync -r {configurations_directory} 'gs://production-data-experiments/storage/experiments/experiments/iprice/configurations/order_acceptance_circuit_breaker_configurations'


ServiceException: 401 Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket.


___

### 4. Uncomment the line below to sync the Presto table.

In [ ]:
# query = """CALL system.sync_partition_metadata('pricing_internal', 'order_acceptance_circuit_breaker_configurations', 'FULL')"""
# df = pd.read_sql(query, presto_connection)
# df

___

### 5. Uncomment the line below to sync the Presto table.

In [ ]:
# query = """SELECT * FROM hive.pricing.order_acceptance_circuit_breaker_configurations"""
# df = pd.read_sql(query, presto_connection)
# df

___